In [ ]:
# Python User-User Collaborative Filtering Recommender System
# consider not using pivot table?
# experiment with different similarity functions

'''
-For each movie j user i has not seen yet, we find the set of users U who are similar to user i and have seen 
movie j.
For each similar user u, we take u's rating of movie j and multiply it by the cosine similarity 
of user i and user u. Sum up these weighted ratings, divide by the number of users in U, and we get a
weighted average rating for the movie j.
'''

In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.preprocessing import normalize

# Read CSV and change to pivot_table
df = pd.read_csv('inputs/boardgame-elite-users.csv')
df = df.pivot_table(index='userID', columns='gameID', values='rating')

# Fill nan with zero. Normalize ignores the zeroes
df = df.fillna(0)
# print(len(df.columns)) --> 402
# print(len(df.index)) --> 199

In [ ]:
# Test / Train Split
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(df)

In [2]:
# Normalize User Ratings for GameIDs
normalized = normalize(df)

In [3]:
# Get User Similarity Matrix
# Also try Pearson coefficient, city-block, etc...
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(df)
# print(sim.shape) --> (199, 199)

In [11]:
# !!! I don't want 'nearest 10 neighbors'...
# I want the nearest 10 neighbors WHO HAVE rated the gameID
# Get K nearest neighbors (indices from user similarity matrix)
# argsort returns indices of the values of a sorted version of the array

# return the 10 nearest neighbors who have rated the game
# should I check to make sure game has not yet been rated?

nbor_indx = sim.argsort()[:,::-1][:,:]

# Takes a gameID and returns list
def get_users_that_rated(gid):
    user_list = df[gid].where(df[gid]>0).dropna().index.tolist()
    return user_list
    
def get_ten_nearest(df, uid, gid):
    pot_nbors = get_users_that_rated(gid)
    uid_indx = df.index.get_loc(uid)
    desc_sim_indx = nbor_indx[uid_indx]
    
    sim_userIDs = []
    for x in desc_sim_indx:
        print(df.index[x])
    
    
print(get_ten_nearest(df,272,3))

272
7100
114589
160442
18639
187094
34384
166746
61548
85009
13340
2044
132526
80659
149084
156708
96817
93535
163106
181472
19677
105883
155092
86405
116385
12305
58562
160824
126571
3080
101462
94593
55101
103398
149815
11829
430
74377
61537
168814
151990
142253
101583
96412
145447
141993
123709
173728
189973
69595
180167
128117
119609
27853
134145
26532
112355
99306
100383
144682
3256
121225
162252
71551
181123
173971
88176
46084
140257
67608
127726
28809
130799
121054
26561
162275
104693
80085
100174
38834
119312
36041
71281
38378
176377
176216
101885
67142
162938
127141
66705
76624
26112
153013
9363
37860
11321
124245
44567
177909
76252
121994
138724
3557
83092
34902
5038
166007
135393
46928
99598
55092
27106
159127
5480
68058
170718
174807
107382
81956
388
118003
49788
144814
131866
44586
137170
145742
132779
95213
180775
86991
31223
112817
64331
164714
173437
80786
70647
175629
13527
99214
153072
181339
161305
58582
176950
142298
66970
57056
119417
35516
12779
16886
33226
122316

In [44]:
print(df.index[1])
y = nbor_indx[0][1]
print(df.index[y])
print(df.index.get_loc(7100))
print(sim[0][10])
print(sim[0][nbor_indx[0][3]])

388
7100
10
0.5953403235711451
0.5803093270234448


In [ ]:
# Prediction Matrix
# For each movie that a user has not rated,
# Get the 10 nearest neighbors that have rated the movie, (how to handle user with no very similar users, every
# nearest neighbor is still more disimilar than similar?)
# Add each rating-Times-Similarity-Weight and divide by 10,
# Of all the new values generated for unrated movies, return the highest N
